In [1]:
import struct
import os
import scipy.io
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout

In [3]:
ECG_data = []

for idx in range(1, 81) : #0~89까지 bin file 순회
    data_path = os.path.join('ECG_' + str(idx) + '.bin') #file 경로 지정
    with open(data_path, "rb") as f : # .bin file을 os 함수를 이용하여 rb로 open
        data = np.frombuffer(f.read(), dtype=np.float16) # bin file 내용 저장
    data = np.append(data, np.array([0]*(5051128 - len(data)))) # 길이가 짧은 것들은 0으로 채우기
    ECG_data.append(data) # dataframe을 만들기 위해 data를 ECG_data에 추가

    f.close() #읽은 .bin file close
    
df_col = [i for i in range(0, 5051128)] #dataframe의 column

ECG = pd.DataFrame(columns = df_col)

for i in range(0, 80) :
    ECG.loc[i] = ECG_data[i]
ECG

,0,1,2,3,4,5,6,7,8,9,...,5051118,5051119,5051120,5051121,5051122,5051123,5051124,5051125,5051126,5051127
0,0.000012,0.002014,0.000856,0.031860,0.000187,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000012,0.002014,0.001469,0.635254,0.000187,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000012,0.002014,0.001469,0.382812,0.000191,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000012,0.002014,0.000061,0.000063,0.000185,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000012,0.002014,0.001713,0.009918,0.000187,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.000012,0.002014,0.019623,0.000155,0.000189,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76,0.000012,0.002014,0.019623,0.000032,0.000191,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,0.000012,0.002014,0.023529,0.504395,0.000184,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,0.000012,0.002014,0.023529,0.000500,0.000187,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
ECG_data = []
sub_num = [1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 36, 38, 39, 41, 42, 43, 44, 45, 47, 48, 52, 53, 54, 55, 59, 59, 60, 62, 63, 64, 65, 66, 68, 70, 72, 73, 74, 75, 76, 77, 78]
for idx in range(1, 81) : #0~89까지 bin file 순회
    if idx in sub_num :
        data_path = os.path.join('ECG_' + str(idx) + '.bin') #file 경로 지정
        with open(data_path, "rb") as f : # .bin file을 os 함수를 이용하여 rb로 open
            data = np.frombuffer(f.read(), dtype=np.float16) # bin file 내용 저장
    
        ECG_data.append(np.array(data))

In [3]:
len(ECG_data)

59

In [5]:
def replace_nan_with_mean(arr):
    nan_indices = np.isnan(arr)
    
    if nan_indices[0]:
        arr[0] = arr[1]
    
    if nan_indices[-1]:
        arr[-1] = (arr[-2] + arr[-3]) / 2
    
    for i in range(1, len(arr) - 1):
        if nan_indices[i]:
            arr[i] = (arr[i-1] + arr[i-2]) / 2
    
    return arr

In [7]:
for i in range(0, len(ECG_data)) :
    ECG_data[i] = replace_nan_with_mean(ECG_data[i])

In [8]:
for i in range(0, len(ECG_data)) :
    print(np.isnan(ECG_data[i]))

[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False

In [18]:
movie_feel = pd.read_csv('./movies.csv')

In [19]:
scores = pd.read_csv('./score.csv')

In [20]:
movie_feel

,First,Second,Third,Fourth,Fifth,Sixth
0,neutral,disgust,fear,sad,happy,anger
1,anger,disgust,neutral,happy,sad,fear
2,happy,neutral,anger,sad,disgust,fear
3,disgust,happy,sad,anger,neutral,fear
4,neutral,anger,disgust,happy,sad,fear
...,...,...,...,...,...,...
84,neutral,sad,happy,fear,disgust,anger
85,anger,sad,neutral,fear,happy,disgust
86,sad,anger,disgust,fear,neutral,happy
87,disgust,sad,anger,happy,fear,neutral


In [21]:
scores.drop([3, 7, 13, 15, 33, 34, 36, 39, 45, 48, 49, 50, 55, 56, 57, 60, 66, 68, 70, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88], axis = 0, inplace = True)

In [22]:
vid1 = scores[['Video1_neutral', 'Video1_fear', 'Video1_sad', 'Video1_happy', 'Video1_anger', 'Video1_disgust']]
vid2 = scores[['Video2_neutral', 'Video2_fear', 'Video2_sad', 'Video2_happy', 'Video2_anger', 'Video2_disgust']]
vid3 = scores[['Video3_neutral', 'Video3_fear', 'Video3_sad', 'Video3_happy', 'Video3_anger', 'Video3_disgust']]
vid4 = scores[['Video4_neutral', 'Video4_fear', 'Video4_sad', 'Video4_happy', 'Video4_anger', 'Video4_disgust']]
vid5 = scores[['Video5_neutral', 'Video5_fear', 'Video5_sad', 'Video5_happy', 'Video5_anger', 'Video5_disgust']]
vid6 = scores[['Video6_neutral', 'Video6_fear', 'Video6_sad', 'Video6_happy', 'Video6_anger', 'Video6_disgust']]
vid1.loc[1].values

array([1, 3, 3, 1, 6, 6], dtype=int64)

In [26]:
matsub_num = ['001', '002', '003', '005', '006', '007', '009', '010', '011', '012', '013', '015', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '036', '038', '039', '041', '042', '043', '044', '045', '047', '048', '052', '053', '054', '055', '059', '060', '062', '063', '064', '065', '066', '068', '070', '072', '073', '074', '075', '076', '077', '078']
len(matsub_num)

59

In [27]:
train_ids = ['001', '005', '006', '009', '010', '011', '012', '013', '015', '021', '022', '023',
             '024', '026', '027', '028', '031', '032', '033', '036', '038', '041', '042', '044',
             '045', '047', '053', '054', '055', '060', '062', '063', '064', '066', '070', '072',
             '073', '075', '076', '077', '078']
test_ids = ['002', '003', '007', '017', '018', '019', '020', '025', '029', '030', '039', '043',
            '048', '052', '059', '065', '068', '074']

In [45]:
train_label = []
test_label = []
for num in matsub_num :
    mat_path = os.path.join('./ECG_experiment_matfile', num, 'selected_data')
    #print(mat_path)
    mat_file = scipy.io.loadmat(mat_path)
    if num in train_ids :
        train_label.append([mat_file['Segment1'][0], vid1.loc[0].values])
        train_label.append([mat_file['Segment2'][0], vid2.loc[0].values])
        train_label.append([mat_file['Segment3'][0], vid3.loc[0].values])
        train_label.append([mat_file['Segment4'][0], vid4.loc[0].values])
        train_label.append([mat_file['Segment5'][0], vid5.loc[0].values])
        train_label.append([mat_file['Segment6'][0], vid6.loc[0].values])
    else :
        test_label.append([mat_file['Segment1'][0], vid1.loc[0].values])
        test_label.append([mat_file['Segment2'][0], vid2.loc[0].values])
        test_label.append([mat_file['Segment3'][0], vid3.loc[0].values])
        test_label.append([mat_file['Segment4'][0], vid4.loc[0].values])
        test_label.append([mat_file['Segment5'][0], vid5.loc[0].values])
        test_label.append([mat_file['Segment6'][0], vid6.loc[0].values])

#x = mat_file['Segment1'].reshape(1, -1)
print(train_label[0])
print(test_label[0])
#len(mat_file['Segment1'][0])

[array([507, 503, 503, ..., 492, 493, 494], dtype=uint16), array([7, 1, 1, 5, 1, 1], dtype=int64)]
[array([445, 444, 446, ..., 453, 454, 453], dtype=uint16), array([7, 1, 1, 5, 1, 1], dtype=int64)]


In [46]:
x_train = []
y_train = []
x_test = []
y_test = []
for xt in train_label :
    print(xt[0].shape)#.reshape(-1).shape)
    x_train.append(xt[0])#tf.convert_to_tensor(xt[0]))
    y_train.append(xt[1])#tf.convert_to_tensor(xt[1]))
    
for xt in test_label :
    print(xt[0].shape)#.reshape(-1).shape)
    x_test.append(xt[0])#tf.convert_to_tensor(xt[0]))
    y_test.append(xt[1])#tf.convert_to_tensor(xt[1]))   

(178490,)
(193736,)
(191559,)
(159628,)
(226382,)
(214049,)
(248543,)
(298048,)
(269761,)
(315226,)
(292998,)
(260666,)
(140709,)
(170159,)
(158161,)
(134164,)
(145618,)
(160889,)
(134351,)
(161111,)
(158381,)
(144727,)
(141451,)
(170396,)
(172779,)
(208135,)
(193459,)
(164107,)
(196794,)
(176780,)
(133879,)
(149905,)
(152488,)
(127160,)
(161277,)
(136982,)
(188084,)
(159751,)
(156134,)
(177836,)
(174822,)
(148297,)
(206909,)
(172541,)
(181659,)
(203402,)
(185867,)
(218833,)
(193425,)
(152510,)
(182887,)
(160569,)
(179787,)
(164289,)
(194011,)
(169678,)
(161786,)
(175598,)
(190724,)
(205193,)
(177450,)
(194191,)
(164728,)
(208924,)
(173434,)
(197540,)
(160676,)
(142946,)
(136297,)
(172865,)
(147932,)
(163445,)
(135853,)
(152703,)
(140593,)
(164288,)
(129535,)
(155336,)
(67616,)
(73388,)
(81084,)
(85757,)
(79709,)
(70914,)
(178702,)
(189001,)
(175674,)
(156895,)
(160530,)
(149020,)
(254153,)
(242331,)
(263018,)
(290600,)
(285675,)
(307347,)
(202610,)
(217980,)
(186541,)
(206102,)
(18025

In [65]:
np.array(y_train[0])
#print(type(x_train[0]))

array([7, 1, 1, 5, 1, 1], dtype=int64)

In [47]:
max_length = 338915
padded_data = np.zeros((len(x_train), max_length))

for i, d in enumerate(x_train):
    padded_data[i, :len(d)] = d

print(padded_data.shape)
x_train = padded_data
print(x_train)

padded_data = np.zeros((len(x_test), max_length))

for i, d in enumerate(x_test):
    padded_data[i, :len(d)] = d

print(padded_data.shape)
x_test = padded_data
print(x_test)

(246, 338915)
[[507. 503. 503. ...   0.   0.   0.]
 [535. 551. 571. ...   0.   0.   0.]
 [500. 500. 498. ...   0.   0.   0.]
 ...
 [593. 606. 612. ...   0.   0.   0.]
 [469. 469. 471. ...   0.   0.   0.]
 [487. 483. 482. ...   0.   0.   0.]]
(108, 338915)
[[445. 444. 446. ...   0.   0.   0.]
 [485. 483. 482. ...   0.   0.   0.]
 [444. 442. 441. ...   0.   0.   0.]
 ...
 [498. 501. 499. ...   0.   0.   0.]
 [486. 488. 488. ...   0.   0.   0.]
 [494. 495. 494. ...   0.   0.   0.]]


In [82]:
print(x_test.shape)

(108, 338915)


In [74]:
for i, idx in enumerate(y_train) :
    #print(i)
    y_train[i] =  np.array(idx)
y_train = np.array(y_train)

for i, idx in enumerate(y_test) :
    #print(i)
    y_test[i] =  np.array(idx)
y_test = np.array(y_test)

In [77]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(max_length, 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy')#,
              #metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
8/8 [==============================] - 56s 7s/step - loss: 195.5749
Epoch 2/10
8/8 [==============================] - 57s 7s/step - loss: 481.7499
Epoch 3/10
8/8 [==============================] - 56s 7s/step - loss: 1355.5248
Epoch 4/10
8/8 [==============================] - 58s 7s/step - loss: 3695.6504
Epoch 5/10
8/8 [==============================] - 57s 7s/step - loss: 8902.8428
Epoch 6/10
8/8 [==============================] - 57s 7s/step - loss: 20761.5820
Epoch 7/10
8/8 [==============================] - 58s 7s/step - loss: 45107.8438
Epoch 8/10
8/8 [==============================] - 59s 7s/step - loss: 95432.0547
Epoch 9/10
8/8 [==============================] - 55s 7s/step - loss: 192856.2656
Epoch 10/10
4/4 [==============================] - 7s 2s/step - loss: 522894.7500


TypeError: cannot unpack non-iterable float object

In [83]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_21 (Conv1D)          (None, 338913, 32)        128       
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 169456, 32)       0         
 g1D)                                                            
                                                                 
 conv1d_22 (Conv1D)          (None, 169454, 64)        6208      
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 84727, 64)        0         
 g1D)                                                            
                                                                 
 conv1d_23 (Conv1D)          (None, 84725, 128)        24704     
                                                                 
 global_average_pooling1d_7   (None, 128)             

In [85]:
cnnoutput = model.predict(x_test)

4/4 [==============================] - 7s 2s/step
